In [ ]:
# Training parameters

window = 672       # 1 week
stride = 4         # 1 hour
latent_dim = 6     # Autoencoder latent dimension
epochs = 150       # Number of epochs
batch_size = 8     # Batch size
f = 7              # Filters' dimensions (conv layers)

In [ ]:
# Building the model

input_shape = X_train.shape[1:]
output_shape = X_train.shape[1:]

#Encoder
encoder_input = tf.keras.Input(shape=input_shape)

x = tfkl.Conv1D(16, f, activation="relu", strides=1, padding="same")(encoder_input)
x = tfkl.MaxPool1D(pool_size=2, strides=2)(x)
x = tfkl.Conv1D(32, f, activation="relu", strides=1, padding="same")(x)
x = tfkl.MaxPool1D(pool_size=2, strides=2)(x)
x = tfkl.Conv1D(64, f, activation="relu", strides=1, padding="same")(x)
x = tfkl.MaxPool1D(pool_size=2, strides=2)(x)

x = tfkl.Flatten()(x)
x = tfkl.Dense(latent_dim, activation='linear')(x)

#Decoder
x = tfkl.Dense(units=42*X_train.shape[2])(x)
x = tfkl.Reshape((42,X_train.shape[2]))(x)
x = tfkl.Conv1DTranspose(64,f,2, padding='same', activation='relu')(x)
x = tfkl.Conv1DTranspose(32,f,2, padding='same', activation='relu')(x)
x = tfkl.Conv1DTranspose(16,f,2, padding='same', activation='relu')(x)
decoder_output = tfkl.Conv1DTranspose(X_train.shape[2],f,2, padding='same', activation='linear')(x)

#Model
autoencoder = tfk.Model(inputs=encoder_input, outputs=decoder_output, name='Autoencoder')
autoencoder.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=[tfk.metrics.MeanSquaredError()])
autoencoder.summary()

In [ ]:
# Training
history = autoencoder.fit(
    X_train,
    X_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data = (X_val,X_val),
    callbacks=[
        tfk.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    ]
).history